In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import re
import csv

import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.corpus import conll2000
nltk.download('conll2000')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize.toktok import ToktokTokenizer

!pip install spacy
!python -m spacy download es_core_news_sm
!pip install --upgrade spacy
import spacy

# Cargar el modelo de lenguaje en español
nlp = spacy.load("es_core_news_sm")


[nltk_data] Downloading package punkt to /Users/sil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sil/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package conll2000 to /Users/sil/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 2.9 MB/s eta 0:00:0000:0100:01


✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [3]:
df = pd.read_csv("../data/capi_chat.csv")
df

,Fecha,Hora,Persona,Mensaje
0,"30/11/23,",19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...
1,"30/11/23,",19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...
2,"30/11/23,",19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.
3,"30/11/23,",19:12:56,Alana Ironhack,Hola
4,"30/11/23,",19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...
...,...,...,...,...
1769,"17/2/24,",21:09:53,Sil🐨,No y que también pensaba que esos días que íba...
1770,"17/2/24,",21:11:04,Alana Ironhack,Jajaj
1771,"17/2/24,",21:11:18,Alana Ironhack,Disfrutar de tu libertad
1772,"17/2/24,",21:18:17,Sil🐨,JAJAJAJAJA


In [4]:
fecha = df['Fecha']

# Eliminar la coma de las fechas
fechas_sin_coma = fecha.str.replace(",", "")

# Reemplazar la columna 'fecha' en el DataFrame original
df['Fecha'] = fechas_sin_coma
df

,Fecha,Hora,Persona,Mensaje
0,30/11/23,19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...
1,30/11/23,19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...
2,30/11/23,19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.
3,30/11/23,19:12:56,Alana Ironhack,Hola
4,30/11/23,19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...
...,...,...,...,...
1769,17/2/24,21:09:53,Sil🐨,No y que también pensaba que esos días que íba...
1770,17/2/24,21:11:04,Alana Ironhack,Jajaj
1771,17/2/24,21:11:18,Alana Ironhack,Disfrutar de tu libertad
1772,17/2/24,21:18:17,Sil🐨,JAJAJAJAJA


In [5]:
# ocultamos los nombres

In [6]:
miembros = df['Persona'].unique()
miembros.shape

(4,)

In [7]:
for i in enumerate(miembros):
    df['Persona'] = df['Persona'].replace(i[1], f'persona {i[0]}')

df

,Fecha,Hora,Persona,Mensaje
0,30/11/23,19:12:50,persona 0,‎Los mensajes y las llamadas están cifrados de...
1,30/11/23,19:12:50,persona 1,‎Alana Ironhack creó el grupo “Capibaras Ironh...
2,30/11/23,19:12:50,persona 0,‎Alana Ironhack te añadió.
3,30/11/23,19:12:56,persona 1,Hola
4,30/11/23,19:13:12,persona 1,Les paso las fotos de ayer prepárense xq es un...
...,...,...,...,...
1769,17/2/24,21:09:53,persona 3,No y que también pensaba que esos días que íba...
1770,17/2/24,21:11:04,persona 1,Jajaj
1771,17/2/24,21:11:18,persona 1,Disfrutar de tu libertad
1772,17/2/24,21:18:17,persona 3,JAJAJAJAJA


In [8]:
# diccionario con el valor y nombre de los dias
week = {
    6:'Domingo',
    0:'Lunes',
    1:'Martes',
    2:'Miercoles',
    3:'Jueves',
    4:'Viernes',
    5:'Sabado'
}

# Diccionario con el valor y nombre de los meses
month = {
    1:'Ene',
    2:'Feb',
    3:'Mar',
    4:'Abr',
    5:'May',
    6:'Jun',
    7:'Jul',
    8:'Ago',
    9:'Sept',
    10:'Obt',
    11:'Nov',
    12:'Dic'
}


# dar formato de fecha a los dias 
df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True)

# nueva columna con los nombres de los dias
#df['Dia'] = df['Fecha'].dt.strftime('%A') #(esto tambien lo cambia)
df['Dia'] = df['Fecha'].dt.weekday.map(week)

# nueva columna con el numero del dia
df['Num_Day'] = df['Fecha'].dt.day

# nueva columna con el año 
df['Year'] = df['Fecha'].dt.year

# nueva columna con el numero de mes
df['Num_Month'] = df['Fecha'].dt.month

# nueva columna con el nombre de los meses
df['Month'] = df['Fecha'].dt.month.map(month)

# organizar las columnas
df = df[['Fecha', 'Dia', 'Num_Day','Num_Month', 'Month', 'Year', 'Hora', 'Persona', 'Mensaje']]

# cambiar el tipo de dato de la columna Day
df['Dia'] = df['Dia'].astype('category')
#cambiar formato fecha 
df['Fecha'] = df['Fecha'].dt.strftime('%d/%m/%Y')
df

,Fecha,Dia,Num_Day,Num_Month,Month,Year,Hora,Persona,Mensaje
0,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 0,‎Los mensajes y las llamadas están cifrados de...
1,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 1,‎Alana Ironhack creó el grupo “Capibaras Ironh...
2,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 0,‎Alana Ironhack te añadió.
3,30/11/2023,Jueves,30,11,Nov,2023,19:12:56,persona 1,Hola
4,30/11/2023,Jueves,30,11,Nov,2023,19:13:12,persona 1,Les paso las fotos de ayer prepárense xq es un...
...,...,...,...,...,...,...,...,...,...
1769,17/02/2024,Sabado,17,2,Feb,2024,21:09:53,persona 3,No y que también pensaba que esos días que íba...
1770,17/02/2024,Sabado,17,2,Feb,2024,21:11:04,persona 1,Jajaj
1771,17/02/2024,Sabado,17,2,Feb,2024,21:11:18,persona 1,Disfrutar de tu libertad
1772,17/02/2024,Sabado,17,2,Feb,2024,21:18:17,persona 3,JAJAJAJAJA


# Cantidad de letras por mensaje

In [9]:
df['Letras'] = df['Mensaje'].apply(lambda s:len(s))

df


,Fecha,Dia,Num_Day,Num_Month,Month,Year,Hora,Persona,Mensaje,Letras
0,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 0,‎Los mensajes y las llamadas están cifrados de...,143
1,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 1,‎Alana Ironhack creó el grupo “Capibaras Ironh...,51
2,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 0,‎Alana Ironhack te añadió.,26
3,30/11/2023,Jueves,30,11,Nov,2023,19:12:56,persona 1,Hola,4
4,30/11/2023,Jueves,30,11,Nov,2023,19:13:12,persona 1,Les paso las fotos de ayer prepárense xq es un...,56
...,...,...,...,...,...,...,...,...,...,...
1769,17/02/2024,Sabado,17,2,Feb,2024,21:09:53,persona 3,No y que también pensaba que esos días que íba...,57
1770,17/02/2024,Sabado,17,2,Feb,2024,21:11:04,persona 1,Jajaj,5
1771,17/02/2024,Sabado,17,2,Feb,2024,21:11:18,persona 1,Disfrutar de tu libertad,24
1772,17/02/2024,Sabado,17,2,Feb,2024,21:18:17,persona 3,JAJAJAJAJA,10


# Cantidad de palabras por mensaje

In [10]:
df['Palabras'] = df['Mensaje'].apply(lambda s:len(s.split(' ')))

df

,Fecha,Dia,Num_Day,Num_Month,Month,Year,Hora,Persona,Mensaje,Letras,Palabras
0,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 0,‎Los mensajes y las llamadas están cifrados de...,143,23
1,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 1,‎Alana Ironhack creó el grupo “Capibaras Ironh...,51,7
2,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 0,‎Alana Ironhack te añadió.,26,4
3,30/11/2023,Jueves,30,11,Nov,2023,19:12:56,persona 1,Hola,4,1
4,30/11/2023,Jueves,30,11,Nov,2023,19:13:12,persona 1,Les paso las fotos de ayer prepárense xq es un...,56,11
...,...,...,...,...,...,...,...,...,...,...,...
1769,17/02/2024,Sabado,17,2,Feb,2024,21:09:53,persona 3,No y que también pensaba que esos días que íba...,57,12
1770,17/02/2024,Sabado,17,2,Feb,2024,21:11:04,persona 1,Jajaj,5,1
1771,17/02/2024,Sabado,17,2,Feb,2024,21:11:18,persona 1,Disfrutar de tu libertad,24,4
1772,17/02/2024,Sabado,17,2,Feb,2024,21:18:17,persona 3,JAJAJAJAJA,10,1


# Metemos la columna de clean_text

In [11]:
df_clean = pd.read_csv('../data/data_capi_chat_clear.csv')

In [19]:
df_clean

,Fecha,Hora,Persona,Mensaje,clean_text
0,"30/11/23,",19:12:50,Capibaras Ironhack,‎Los mensajes y las llamadas están cifrados de...,mensaje llamada cifrado extremo extremo nadie ...
1,"30/11/23,",19:12:50,Alana Ironhack,‎Alana Ironhack creó el grupo “Capibaras Ironh...,aln ironhack creer grupo capibara ironhack
2,"30/11/23,",19:12:50,Capibaras Ironhack,‎Alana Ironhack te añadió.,aln ironhack t anadio
3,"30/11/23,",19:12:56,Alana Ironhack,Hola,holar
4,"30/11/23,",19:13:12,Alana Ironhack,Les paso las fotos de ayer prepárense xq es un...,l pasar foto ayer preparense ser bombardeo
...,...,...,...,...,...
1769,"17/2/24,",21:09:53,Sil🐨,No y que también pensaba que esos días que íba...,tambien pensar dia ibir hacer
1770,"17/2/24,",21:11:04,Alana Ironhack,Jajaj,jajaj
1771,"17/2/24,",21:11:18,Alana Ironhack,Disfrutar de tu libertad,disfrutar libertad
1772,"17/2/24,",21:18:17,Sil🐨,JAJAJAJAJA,jajajajajar


In [22]:
# Unir los DataFrames basándote en la columna común
df_resultado = pd.merge(df, df_clean[['Mensaje', 'clean_text']], on='Mensaje')

df_resultado

,Fecha,Dia,Num_Day,Num_Month,Month,Year,Hora,Persona,Mensaje,Letras,Palabras,clean_text
0,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 0,‎Los mensajes y las llamadas están cifrados de...,143,23,mensaje llamada cifrado extremo extremo nadie ...
1,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 1,‎Alana Ironhack creó el grupo “Capibaras Ironh...,51,7,aln ironhack creer grupo capibara ironhack
2,30/11/2023,Jueves,30,11,Nov,2023,19:12:50,persona 0,‎Alana Ironhack te añadió.,26,4,aln ironhack t anadio
3,30/11/2023,Jueves,30,11,Nov,2023,19:12:56,persona 1,Hola,4,1,holar
4,30/11/2023,Jueves,30,11,Nov,2023,19:13:12,persona 1,Les paso las fotos de ayer prepárense xq es un...,56,11,l pasar foto ayer preparense ser bombardeo
...,...,...,...,...,...,...,...,...,...,...,...,...
3849,17/02/2024,Sabado,17,2,Feb,2024,20:59:23,persona 1,No te contratamos,17,3,t contratamos
3850,17/02/2024,Sabado,17,2,Feb,2024,21:09:29,persona 3,Ya joo JAJAJAJAJA,17,3,joo jajajajajar
3851,17/02/2024,Sabado,17,2,Feb,2024,21:09:53,persona 3,No y que también pensaba que esos días que íba...,57,12,tambien pensar dia ibir hacer
3852,17/02/2024,Sabado,17,2,Feb,2024,21:11:18,persona 1,Disfrutar de tu libertad,24,4,disfrutar libertad


# De qué se hablaba en el grupo los dias de mayor actividad?

In [1]:
# falta instalar la libreria
for i in range(5):
    # seleccion de fecha
    fecha = Daily_LineTime_Sort.Date[i]
    
    # seleccion de mensajes con solo texto de la fecha dada
    df_fecha = df[(df['Date'] == fecha) & (df['Message'])]
    
    # nube de palabras
    Nube_Words(df_fecha, fecha)

NameError: name 'Daily_LineTime_Sort' is not defined